In [17]:
import requests
from bs4 import BeautifulSoup
import json

# Constantes
BASE_URL = "https://www.amazon.es/s?k="

# Función para obtener el contenido HTML de una página usando ScraperAPI
def obtener_soup(api_key, url):
    params = {
        'api_key': api_key,
        'url': url,
        'country_code': 'ES'  # Código de país para España
    }

    response = requests.get('https://api.scraperapi.com/', params=params)

    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f'Error al obtener la página: {response.status_code}')
        return None

# Función para obtener los links de productos en una página
def obtener_links_productos(soup):
    links = []
    productos = soup.find_all('a', {'class': 'a-link-normal'})
    for producto in productos:
        href = producto.get('href')
        if href and '/dp/' in href:
            links.append("https://www.amazon.es" + href)
    return links

# Función para obtener los IDs de los productos a partir de los links
def obtener_ids_productos(links):
    ids_productos = set()  # Usamos un set para evitar duplicados
    for link in links:
        product_id = link.split('/dp/')[1].split('/')[0]
        ids_productos.add(product_id)  # Agregar al set, los duplicados se eliminan automáticamente
    return ids_productos

# Función principal para recolectar los IDs de productos
def recolectar_ids(api_key, query):
    url_base = BASE_URL + query
    page_number = 1
    ids_productos = set()  # Usamos un set para evitar duplicados

    while True:
        print(f"Obteniendo productos de la página {page_number}...")
        url_pagina = f"{url_base}&page={page_number}"
        soup = obtener_soup(api_key, url_pagina)
        
        if not soup:
            print(f"No se pudo obtener la página {page_number}. Terminando la recolección.")
            break

        # Obtener los links de productos de la página
        links_productos = obtener_links_productos(soup)
        if not links_productos:
            print(f"No se encontraron productos en la página {page_number}. Terminando la recolección.")
            break

        # Obtener los IDs de los productos y agregar al set
        ids_pagina = obtener_ids_productos(links_productos)
        ids_productos.update(ids_pagina)  # Usamos .update() para agregar sin duplicados

        # Aumentar el número de página
        page_number += 1

    # Filtrar los IDs para eliminar los que tengan más de 10 caracteres
    ids_productos_filtrados = [id for id in ids_productos if len(id) <= 10]

    # Guardar los IDs filtrados en un archivo JSON
    with open("ids.json", "w") as f:
        json.dump(ids_productos_filtrados, f, indent=4)

    print(f"Se guardaron {len(ids_productos_filtrados)} IDs de productos en 'ids.json'.")

# Ejecución del programa
if __name__ == "__main__":
    API_KEY = '83bc7b5f85e1d83b695e0d39816ebb77'  # Tu API Key
    query = "moviles"  # Tu consulta de búsqueda
    
    recolectar_ids(API_KEY, query)


Obteniendo productos de la página 1...
Obteniendo productos de la página 2...
Obteniendo productos de la página 3...
Obteniendo productos de la página 4...
Obteniendo productos de la página 5...
Obteniendo productos de la página 6...
Obteniendo productos de la página 7...
Obteniendo productos de la página 8...
Obteniendo productos de la página 9...
Obteniendo productos de la página 10...
Obteniendo productos de la página 11...
Obteniendo productos de la página 12...
Obteniendo productos de la página 13...
No se encontraron productos en la página 13. Terminando la recolección.
Se guardaron 503 IDs de productos en 'ids.json'.


Porque va amas paguinas de las que existen?
Porque hay que tienen mayor de 10?
porque hay repetidos?